# Caso práctico 3 


En la primera parte, el programa busca que tengas los archivos y estos se descarguen para poder trabajar con ellos, se edita su primera variable, ya que mis archivos son locales y no descargadas de la web, se usan diveras funciones en fin de que tengamos nuestra base de datos lista para manipularse.
Primero se combinan mis archivos locales y se reparan, para posteriormente eliminar los archivos zip orginales combinados, louego comenzamos a descompromimir el archivo que ya habíamos reparado.
En las últimas 2 líneas, se cuentan los archivos descomprimidos y se instala el hdf5storage.

In [ ]:
!cat "C:/Users/DELL/Downloads/stage1_train.zip" "C:/Users/DELL/Downloads/stage1_test.zip" "C:/Users/DELL/Downloads/stage2_test_final.zip" > combined_temp.zip
!zip -FF combined_temp.zip --out data.zip
!rm "C:/Users/DELL/Downloads/stage1_train.zip" "C:/Users/DELL/Downloads/stage1_test.zip" "C:/Users/DELL/Downloads/stage2_test_final.zip"
!unzip data.zip -d data && rm data.zip
!ls data | wc -l
!pip install hdf5storage

Se hacen las importaciones, de librerías hy funciones que nos ayudan con el procesamiento de datos y manejo de los argumentos a trabajar, creando igualmente una excepcion oara en caso de que no se enecuentre el archivo, creando una función def que solucione lo que courre enncaso de no poder localizar el archivo.

In [ ]:
import os
import argparse
import sys
import numpy as np
import hdf5storage
import cv2
import matplotlib.pyplot as plt 

class NoDataFound(Exception):
    pass

def dir_path(path):
    """Check the path and the existence of a data directory"""
    path = path.replace('\\', '/')
    data_path = os.path.join(path, 'data').replace('\\', '/')
    if os.path.isdir(data_path):
        return path
    elif os.path.isdir(path):
        raise NoDataFound('Could not find a "data" folder inside directory. {} does not exist.'
                          .format(data_path))
    else:
        raise NotADirectoryError(path)
        

### Creacion de imagenes, etiquetas y máscaras de matrices numpy

Después se manda al directorio correcto en donde tengo mis archivos locales; para luego proseguir con el proceso de creación de imagenes, etiquetas y las matrices. 
En un principio, hice la creación de las nuevas clases, ya con nuestro porpio dataset adaptado, después hice un gráfico de barras capaz de mostrar la distribución.
Para luego hacer una visualización de las primeras tres imágenes junto con sus máscaras y las máscaras superpuestas en las imágenes originales.

In [ ]:
integer_to_class = {'1': 'Humano', '2': 'Ratón', '3': 'Mosca'}

labels = np.load('labels.npy')
images = np.load('images.npy')
masks = np.load('masks.npy')

classes, counts = np.unique(labels, return_counts=True)
plt.bar(classes, counts, tick_label=[integer_to_class[str(cls)] for cls in classes])
for i, idx in enumerate(classes):
    print('number of {}: {}'.format(integer_to_class[str(idx)], counts[i]))

plt.figure(figsize=(12, 10))
for i in range(3):
    plt.subplot(3, 3, 1 + i * 3)
    plt.title('Input')
    plt.imshow(images[i, :])
    plt.subplot(3, 3, 2 + i * 3)
    plt.title('Mask')
    plt.imshow(masks[i, :])
    plt.subplot(3, 3, 3 + i * 3)
    plt.title('Detection')
    plt.imshow(images[i, :], cmap='gray')
    mask = np.ma.masked_where(masks[i] == False, masks[i])
    plt.imshow(mask, alpha=0.8, cmap='Set1')


Luego de ello, se generaron 18 imágenes aleatorias para ver la diversidad y precisión del dataset.

In [ ]:
plt.figure(figsize=(16, 8))
for i, idx in enumerate(np.random.randint(images.shape[0], size=18), start=1):
    plt.subplot(3, 6, i)
    plt.imshow(images[idx], cmap='gray')
    mask = np.ma.masked_where(masks[idx] == False, masks[idx])
    plt.imshow(mask, alpha=0.8, cmap='Set1')
    plt.title(integer_to_class[str(labels[idx])])
    plt.axis('off')

### Delimitación y redimensionamiento de imagenes

Uso de funciones y variables para acomodar y acoplar las imagenes ya obtenidas.
La primera es una función get_bounding_box y la segunda es crop_to_bbox.

In [ ]:
def get_bounding_box(mask):
    """Return the bounding box of a mask image."""
    xmin, ymin, xmax, ymax = 0, 0, 0, 0
    for row in range(mask.shape[0]):
        if mask[row, :].max() != 0:
            ymin = row
            break
    for row in range(mask.shape[0] - 1, -1, -1):
        if mask[row, :].max() != 0:
            ymax = row
            break
    for col in range(mask.shape[1]):
        if mask[:, col].max() != 0:
            xmin = col
            break
    for col in range(mask.shape[1] - 1, -1, -1):
        if mask[:, col].max() != 0:
            xmax = col
            break
    return xmin, ymin, xmax, ymax


In [ ]:
def crop_to_bbox(image, bbox, crop_margin=10):
    """Crop an image to the bounding by forcing a squared image as output."""
    x1, y1, x2, y2 = bbox
    max_width_height = np.maximum(y2 - y1, x2 - x1)
    y2 = y1 + max_width_height
    x2 = x1 + max_width_height
    y1 = np.maximum(y1 - crop_margin, 0)
    y2 = np.minimum(y2 + crop_margin, image.shape[0])
    x1 = np.maximum(x1 - crop_margin, 0)
    x2 = np.minimum(x2 + crop_margin, image.shape[1])
    return image[y1:y2, x1:x2]


Luego de ello, se hace un proceso de recorte y delimitación de las imagenes, comenzando por recortarlas a su cuadro delimitador y redimensionarlas a un tamaño uniforme, para después visualizar el resultado. 

In [ ]:
from IPython.display import display, clear_output

dim_cropped_image = 224
images_cropped = []
for i in range(images.shape[0]):
    if i % 10 == 0:
        clear_output(wait=True)
        display('[{}/{}] images processed: {:.1f} %'.format(i+1, images.shape[0], (i+1) / images.shape[0] * 100))
    bbox = get_bounding_box(masks[i])
    image = crop_to_bbox(images[i], bbox, 20)
    image = cv2.resize(image, dsize=(dim_cropped_image, dim_cropped_image), interpolation=cv2.INTER_CUBIC)
    images_cropped.append(image)

clear_output(wait=True)
display('[{}/{}] images processed: {:.1f} %'.format(i+1, images.shape[0], (i+1) / images.shape[0] * 100))

images_cropped = np.array(images_cropped)
print(images_cropped.shape)

plt.figure(figsize=(16, 8))
for i, idx in enumerate(np.random.randint(images_cropped.shape[0], size=18), start=1):
    plt.subplot(3, 6, i)
    plt.imshow(images_cropped[idx], cmap='gray')
    plt.title(integer_to_class[str(labels[idx])])
    plt.axis('off')


Importaremos los modulos necesarios para trabajar con imagenes y cargaremos los datos, para después volver a redimensionar las diversas imagenes que ya obtuvimos anteriormente. 
Al final se recorre cada imagen en el dataset y se normaliza, es decir, se escalan los píxeles para que estén en el rango [0, 1].

In [ ]:
import os
import argparse
import sys
import numpy as np
import hdf5storage
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


In [ ]:
labels = np.load('labels.npy')
images = np.load('images.npy')
masks = np.load('masks.npy')


In [ ]:
images = images[..., np.newaxis]


In [ ]:
for i in range(images.shape[0]):
    images[i, :, :] = (images[i, :, :] - np.min(images[i, :, :])) / (np.max(images[i, :, :]) - np.min(images[i, :, :]))


### Métricas de evaluación 
dice_coef, dice_coef_loss, bce_dice_loss: Definen funciones de pérdida y métrica personalizada para el modelo U-Net.

In [ ]:
from keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

smooth = 1

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1.-dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_coef_loss(y_true, y_pred)


### División de datos en entrenamiento, validación y prueba:

Divide el conjunto de datos en tres partes: entrenamiento, validación y prueba.

In [ ]:
images_train, images_test, masks_train, masks_test = train_test_split(images, masks, test_size=0.2, train_size=0.8, random_state=1)
images_test, images_cv, masks_test, masks_cv = train_test_split(images_test, masks_test, test_size=0.5, train_size=0.5, random_state=1)

print("number of training examples = " + str(images_train.shape[0]))
print("number of development examples = " + str(images_cv.shape[0]))
print("number of test examples = " + str(images_test.shape[0]))
print("Images_train shape: " + str(images_train.shape))
print("Masks_train shape: " + str(masks_train.shape))
print("Images_val (dev) shape: " + str(images_cv.shape))
print("Masks_val (dev) shape: " + str(masks_cv.shape))
print("Images_test shape: " + str(images_test.shape))
print("Masks_test shape: " + str(masks_test.shape))


### Definición del modelo U-Net:
Construcción del modelo U-Net para la segmentación de imágenes.

In [ ]:
def unet(input_size=(224, 224, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv1[_{{{CITATION{{{_1{](https://github.com/TannerFry/CS525-Final-Project/tree/972297369dc9af46c22ce2ad552e470c219fc038/extract_crops.py)[_{{{CITATION{{{_2{](https://github.com/haboub4500/Segmentation-of-biological-cells/tree/dce03362feb151ae7186bf592c5022322edbd2ac/model.py)[_{{{CITATION{{{_3{](https://github.com/johnmattox/TCC_Pait/tree/4f49b0a5c71ae2e33a73f0fa46bf283a780b1903/U-NET%2Ftrain_unet.py)[_{{{CITATION{{{_4{](https://github.com/DomSas/2D-unet/tree/390e23abd13d8e41e71a2f33693de24b5819bd7a/server_code_model_mar_2D.py)[_{{{CITATION{{{_5{](https://github.com/rciric/deep-generative-connectome/tree/aad6b95130306dd00ad23f493f21e33b93bf6aa4/connectogen%2Fmodels%2Fvae%2Fsubtle_metrics.py)[_{{{CITATION{{{_6{](https://github.com/EvergreenTree/mosinska2018beyond/tree/3612abde89f6de87ed3e16e933d4ed1d4092234d/unet_server.py)[_{{{CITATION{{{_7{](https://github.com/selamgit/segmentation_with_U-net/tree/075c0eb7045d21473b28e2f9af045d04a2dac4bd/model.py)[_{{{CITATION{{{_8{](https://github.com/jaybo/model_crack_detect/tree/d21e380706f33ca6007600292a4d009c5a6c61c5/models.py)